In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds
import tensorflow.keras as keras
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
train, info = tfds.load(
    'lfw',
    split='train',
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
# Converting from tf dataset to numpy arrays because of the label-image inversion problem.
train_X = np.zeros((len(list(train)), 250, 250, 3))
train_y = list()

for i, element in enumerate(tqdm(train)):
    train_X[i] = element[1].numpy().astype(np.int32) / 255
    train_y.append(element[0].numpy().decode('UTF-8'))

In [ ]:
# Callback for early stopping
es_callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=5,
    restore_best_weights=True
)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu', input_shape=(256, 256, 3)),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),
    keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    keras.layers.MaxPooling2D(pool_size=2),
    keras.layers.Dropout(0.3),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation=None),
    keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1))
], name='zeiler_fergus')

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0001),
    loss=tfa.losses.TripletSemiHardLoss(),
    metrics=['accuracy']
)

In [ ]:
train_y = LabelEncoder().fit_transform(train_y)

In [ ]:
history = model.fit(
    train_X,
    train_y,
    epochs=5,
    verbose=5
)